Given $N \in \mathbb{N}$ and $D > 0$, we create a point set on the plane $U \subset \mathbb{R}^2$ of size $N$ in the range $[-D, D]$.
We create a graph using based on $U$ with parameter $c, (c_1, c_2)$.
The graph is created by first generating the Delaunay mesh from $U$, then we add $c$ additional edges but also removing intersecting edges from the original Delaunay.
The number of intersecting edges stays in the range $[c_1, c_2]$. 

In [1]:
from mapmatch.input_graph import create_points, create_graph_from_points

U = create_points(n=16, d=1)

P0, _ = create_graph_from_points(U, remove_count=0, remove_threshold_interval=(0, 0))
P1, d1 = create_graph_from_points(U, remove_count=1, remove_threshold_interval=(0, 3))
P2, d2 = create_graph_from_points(U, remove_count=1, remove_threshold_interval=(7, 100))

Now we will visualize the graphs by showing vertices and edges. And show the additional added and removed edges that are green and red respectively.

In [2]:
from mapmatch.visualisation import visualize_graph, visualize_graph_debug

visualize_graph(P0, "graph-0")
visualize_graph_debug(P1, d1, "graph-1")
visualize_graph(P1, "graph-2")
visualize_graph_debug(P2, d2, "graph-3")
visualize_graph(P2, "graph-4")

Now we will add the highway graph. We have three parameters.
The $h \in \mathbb{N}$ parameter represents the number of highway points that needs to be divisible by 4.
Because each highway lane needs $h/4$ points. The $h_v > 0$ parameter represents the distance between two highway lanes.
And lastly the exit list $h_i \in \mathbb{N} $ with numbers ranging $0 \leq h_i \leq h$ represents points of the highway that merges with the original graph.

In [ ]:
from mapmatch.input_graph import create_graph_highway_from_graph

U2 = create_points(n=16, d=1)
P_original, _ = create_graph_from_points(U2, remove_count=3, remove_threshold_interval=(1, 4))
P_highway = create_graph_highway_from_graph(P_original, d=1, highway_points=6*4, highway_thickness=0.1, exits=[2, 6+4, 12+2, 18+4])

Now we will visualize the newly created highway. The black lines represent the original Delaunay based mesh. The red lines are the highways, and the green lines connect the Delaunay based mesh with the highway, that is the exits of the highways.

In [4]:
from mapmatch.visualisation import visualize_highway_debug

visualize_graph(P_original, "graph-1-0")
visualize_highway_debug(P_original, P_highway, "graph-1-1")
visualize_graph(P_highway, "graph-1-2")

Now apply the same principle for bigger highway graphs.

In [5]:
from mapmatch.input_graph import graph_largest_component, GraphNeighbour

U3 = create_points(n=128, d=1)
P_original2, _ = create_graph_from_points(U3, remove_count=36, remove_threshold_interval=(3, 12))
P_original2 = graph_largest_component(P_original2, GraphNeighbour(P_original2))
P_highway2 = create_graph_highway_from_graph(P_original2, d=1, highway_points=18*4, highway_thickness=0.1, exits=[2, 18+12, 36+5, 54+17])

visualize_graph(P_original2, "graph-2-0", 0.5)
visualize_highway_debug(P_original2, P_highway2, "graph-2-1", 0.5)
visualize_graph(P_highway2, "graph-2-2", 0.5)

Finally, we will create four graphs: tiny, small, medium and big.
These four graphs will be stored with pickle, so users can reproduce results from the thesis.
The file saving lines are commented out, to stay consistent with the thesis result input.

In [ ]:
import pickle
from mapmatch.input_graph import graph_largest_component, GraphNeighbour, graph_pertube

U_tiny = create_points(n=12, d=1)
U_small = create_points(n=16, d=1)
U_medium = create_points(n=48, d=1)
U_large = create_points(n=128, d=1)

P_original_tiny, _ = create_graph_from_points(U_small, remove_count=2, remove_threshold_interval=(1, 5))
P_original_small, _ = create_graph_from_points(U_small, remove_count=3, remove_threshold_interval=(2, 5))
P_original_medium, _ = create_graph_from_points(U_medium, remove_count=8, remove_threshold_interval=(4, 8))
P_original_large, _ = create_graph_from_points(U_large, remove_count=12, remove_threshold_interval=(9, 14))

P_original_tiny = graph_largest_component(P_original_tiny, GraphNeighbour(P_original_tiny))
P_original_small = graph_largest_component(P_original_small, GraphNeighbour(P_original_small))
P_original_medium = graph_largest_component(P_original_medium, GraphNeighbour(P_original_medium))
P_original_large = graph_largest_component(P_original_large, GraphNeighbour(P_original_large))

P_highway_tiny = P_original_tiny
P_highway_small = create_graph_highway_from_graph(P_original_small, d=1, highway_points=4*4, highway_thickness=0.1, exits=[0, 4+1, 8+2, 12+3])
P_highway_medium = create_graph_highway_from_graph(P_original_medium, d=1, highway_points=4*4, highway_thickness=0.1, exits=[0, 4+1, 8+2, 12+3])
P_highway_large = create_graph_highway_from_graph(P_original_large, d=1, highway_points=6*4, highway_thickness=0.1, exits=[0, 6+2, 12+3, 18+5])

P_highway_tiny = graph_pertube(P_highway_tiny, 1e-10, 1024)
P_highway_small = graph_pertube(P_highway_small, 1e-10, 1024)
P_highway_medium = graph_pertube(P_highway_medium, 1e-10, 1024)
P_highway_large = graph_pertube(P_highway_large, 1e-10, 1024)

# pickle.dump(P_highway_tiny, open("highway_tiny.dat", "wb"))
# pickle.dump(P_highway_small, open("highway_small.dat", "wb"))
# pickle.dump(P_highway_medium, open("highway_medium.dat", "wb"))
# pickle.dump(P_highway_large, open("highway_large.dat", "wb"))

In [7]:
P_highway_tiny = pickle.load(open("highway_tiny.dat", "rb"))
P_highway_small = pickle.load(open("highway_small.dat", "rb"))
P_highway_medium = pickle.load(open("highway_medium.dat", "rb"))
P_highway_large = pickle.load(open("highway_large.dat", "rb"))

visualize_graph(P_highway_tiny, "graph-tiny", 1)
visualize_graph(P_highway_small, "graph-small", 1)
visualize_graph(P_highway_medium, "graph-medium", 0.5)
visualize_graph(P_highway_large, "graph-large", 0.25)